In [61]:
import re
import math
import numpy as np
import nltk
import copy
from string import digits
from nltk.stem import PorterStemmer
from os import listdir

In [ ]:
class MaxPriorityQueue:
    # Initialize priority queue
    def __init__(self, items=None): 
        self.heap = items or []
        if self.heap:
            for i in reversed(range(len(self.heap) // 2)):
                self._sift_down(i)
        self.position_map = {tuple(item): idx for idx, item in enumerate(self.heap)}

    # Insert one element
    def insert(self, item):
        self.heap.append(item)
        self.position_map[tuple(item)] = len(self.heap) - 1
        self._sift_up(len(self.heap) - 1)

    # Get the maximum value
    def pop(self):
        if self.is_empty():
            raise IndexError("Pop from an empty priority queue")
        self._swap(0, len(self.heap) - 1)
        item = self.heap.pop()
        del self.position_map[tuple(item)]
        self._sift_down(0)
        return item
    # Check the maximum value
    def peek(self):
        if self.is_empty():
            return None
        return self.heap[0]

    # Delete specific element
    def remove(self, item):
        idx = self.position_map.pop(tuple(item), None)
        if idx is None:
            raise ValueError("Item not found in the priority queue")

        if idx == len(self.heap) - 1:
            self.heap.pop()
        else:
            self._swap(idx, len(self.heap) - 1)
            self.heap.pop()
            self._sift_down(idx)
            self._sift_up(idx)

    # Deep copy
    def copy(self):
        return MaxPriorityQueue(self.heap[:])
    
    # Check if queue is empty
    def is_empty(self):
        return len(self.heap) == 0

    # Get the number of elements in queue
    def size(self):
        return len(self.heap)

    # Adjust the heap
    def _sift_up(self, index):
        parent = (index - 1) // 2
        if parent >= 0 and self.heap[index][0] > self.heap[parent][0]:
            self._swap(index, parent)
            self._sift_up(parent)

    def _sift_down(self, index):
        left = 2 * index + 1
        right = 2 * index + 2
        largest = index

        if left < len(self.heap) and self.heap[left][0] > self.heap[largest][0]:
            largest = left
        if right < len(self.heap) and self.heap[right][0] > self.heap[largest][0]:
            largest = right

        if largest != index:
            self._swap(index, largest)
            self._sift_down(largest)

    # Swap two element in the heap
    def _swap(self, i, j):
        self.position_map[tuple(self.heap[i])] = j
        self.position_map[tuple(self.heap[j])] = i
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

In [6]:
FILE_PATH = "./data/"

ps = PorterStemmer()

r = open("stopwords.txt")
stopwords = r.read()

In [7]:
def preprocessing(file):
    # Preprocessing
    doc = file.replace("\s+"," ").replace("\n", "").replace("\r\n", "")
    doc = re.sub(r"[^\w\s]", "", doc)
    doc = re.sub(r"[0-9]", "", doc)
    doc = doc.replace("_", "")

    # Tokenization and Lowercasing
    tokenization = [word.lower() for word in doc.split(" ")]

    # Stemming using Porter's Algorithm
    stemming = [ps.stem(word) for word in tokenization]

    # Stopword Removal
    result = [word for word in stemming if word not in stopwords]

    return result

In [8]:
def count_tf_df(doc_set):
    tf_all = list()
    df_all = dict()

    for document in doc_set:
        doc_id, doc = document

        token_list = preprocessing(doc)
        
        tf = dict()
        for term in token_list:
            if term in tf:
                tf[term] += 1
            else:
                tf[term] = 1
        tf_all.append([doc_id, tf])

        for term in tf:
            if term in df_all:
                df_all[term] += 1
            else:
                df_all[term] = 1

    df_all = dict(sorted(df_all.items(), key=lambda x: x[0]))

    term_index = dict()
    index = 0
    for term in df_all:
        term_index[term] = index
        index += 1

    return tf_all, df_all, term_index    

In [9]:
def tf_vec(tf_list, t_index):
    tf_vector = list()

    for pair in tf_list:
        doc_id, tf = pair
        vec = np.array([0] * len(t_index), dtype=float)
        
        for word in tf:
            vec[t_index[word]] = tf[word]
        
        tf_vector.append([doc_id, vec])
        
    return tf_vector

In [10]:
def tf_idf_vec(tf_vector, df_list, t_index):
    
    idf_vector = np.array([0] * len(t_index), dtype=float)

    N = len(tf_vector)
   
    for word, df in df_list.items():
        idf = math.log(N / df, 10)
        idf_vector[t_index[word]] = idf

    tf_idf_vectors = list()
    for vec in tf_vector:
        index = vec[0]
        tf_idf = vec[1] * idf_vector
        tf_idf_unit = tf_idf / np.linalg.norm(tf_idf)
        tf_idf_vectors.append([index, tf_idf_unit])
        
    return tf_idf_vectors


In [ ]:
files = listdir(FILE_PATH)
files.sort(key=lambda x: int(x[:-4]))
doc_set = list()
doc_tf_idf = list()

for file in files:
    with open(FILE_PATH + file, "r") as f:
        document_id = str(file)[:-4]
        document = f.read()
        doc_set.append([document_id, document])

tf_list, df_list, t_index = count_tf_df(doc_set)

tf_vector = tf_vec(tf_list, t_index)
tf_idf_vector = tf_idf_vec(tf_vector, df_list, t_index)

for vector in tf_idf_vector:
    doc_id, vec_list = vector
    terms_num = np.count_nonzero(vec_list)
    cur_tf_idf = list()
    for i in range(len(vec_list)):
        if vec_list[i] != 0:
            cur_tf_idf.append([i,vec_list[i]])
    doc_tf_idf.append(cur_tf_idf)

In [ ]:
def get_vector(doc_id, t_index):
    vector = np.array([0] * len(t_index), dtype=float)
    count = 0
    for term in doc_tf_idf[doc_id-1]:
        count += 1
        vector[term[0]] = term[1] 
    return vector

def cosine(doc_x, doc_y):
    vector_x = get_vector(doc_x, t_index)
    vector_y = get_vector(doc_y, t_index)
    cosine_sim = float(np.dot(vector_x, vector_y))
    return cosine_sim

In [ ]:
# Initialize cosine similarity and priority queue
def pre_clustering(doc_set):
    N = len(doc_set)
    cos_i_j = [[] for _ in range(N)]
    prior = []   

    for i in range(N):
        pq = MaxPriorityQueue()
        for j in range(N):
            sim = cosine(i + 1, j + 1)
            cos_i_j[i].append([sim, j])
            if i != j:
                pq.insert(cos_i_j[i][j])
        prior.append(pq)
    return cos_i_j, prior

In [ ]:
# HAC algorithm
def HAC(N, cos_i_j, prior):
    alive = [1] * N
    merge = []

    # Clustering for N-1 round
    for _ in range(N - 1):
        max_sim = -1
        k1 = k2 = -1

        # Find the maximum cosine similarity
        for i in range(N):
            if alive[i] == 1 and not prior[i].is_empty():
                top = prior[i].peek()
                if top[0] > max_sim:
                    max_sim = top[0]
                    k1, k2 = i, top[1]

        # Record the merge docs
        merge.append([k1 + 1, k2 + 1])
        
        # Merge k2 into k1
        alive[k2] = 0
        prior[k1] = MaxPriorityQueue()
        
        # Update cosine similarity of every cluster
        for i in range(N):
            if alive[i] == 1 and i != k1:
                prior[i].remove(cos_i_j[i][k1])
                prior[i].remove(cos_i_j[i][k2])
                # Update cosine similarity for the new cluster
                new_sim = max(cosine(i + 1, k1 + 1), cosine(i + 1, k2 + 1))
                cos_i_j[i][k1][0] = new_sim
                prior[i].insert(cos_i_j[i][k1])

                cos_i_j[k1][i][0] = new_sim
                prior[k1].insert(cos_i_j[k1][i])
    return merge

In [62]:
cos_i_j, prior = pre_clustering(doc_set)

In [101]:
cos_sim = copy.deepcopy(cos_i_j)
pq_list = copy.deepcopy(prior)

In [ ]:
merge_result = HAC(len(doc_set), cos_sim, pq_list)

[[48, 49], [527, 529], [662, 663], [621, 622], [732, 733], [848, 849], [8, 9], [195, 229], [195, 230], [305, 309], [476, 477], [564, 565], [564, 595], [564, 596], [705, 706], [101, 106], [926, 928], [943, 944], [211, 212], [211, 213], [792, 796], [191, 192], [1034, 1036], [500, 501], [832, 838], [155, 158], [676, 677], [507, 508], [855, 856], [583, 584], [243, 244], [681, 684], [1085, 1088], [822, 824], [330, 333], [198, 200], [896, 897], [100, 102], [197, 202], [605, 607], [520, 523], [491, 493], [825, 828], [736, 737], [899, 906], [499, 500], [841, 842], [372, 381], [968, 969], [54, 55], [839, 840], [88, 89], [911, 912], [821, 822], [970, 971], [546, 547], [954, 955], [1000, 1008], [231, 235], [885, 886], [996, 997], [109, 110], [94, 98], [719, 722], [56, 57], [245, 250], [502, 503], [839, 841], [130, 131], [327, 328], [1084, 1085], [531, 532], [197, 199], [302, 313], [968, 970], [455, 456], [922, 924], [615, 616], [281, 282], [1041, 1042], [598, 602], [34, 35], [574, 575], [507, 512

In [ ]:
# Output
t_result = [[i] for i in range(1, len(doc_set)+1)]
for i in range(len(doc_set)-8):
    for j in range(len(t_result[merge_result[i][1]-1])):
        t_result[merge_result[i][0]-1].append(t_result[merge_result[i][1]-1][j])

    if i == len(doc_set)-20:
        with open(f"./20.txt", "w") as f:
            for j in range(len(doc_set)):
                if len(t_result[j]) != 0:
                    t_result[j].sort()
                    for k in range(len(t_result[j])):
                        f.write(f"{t_result[j][k]}\n")
                    f.write(f"\n")
        t_result[merge_result[i][1]-1].clear()

    elif len(doc_set)-13:
        with open(f"./13.txt", "w") as f:
            for j in range(len(doc_set)):
                if len(t_result[j]) != 0:
                    t_result[j].sort()
                    for k in range(len(t_result[j])):
                        f.write(f"{t_result[j][k]}\n")
                    f.write(f"\n")
        t_result[merge_result[i][1]-1].clear()

    t_result[merge_result[i][1]-1].clear()


with open(f"./8.txt", "w") as f:
    for j in range(len(doc_set)):
        if len(t_result[j]) != 0:
            t_result[j].sort()
            for k in range(len(t_result[j])):
                f.write(f"{t_result[j][k]}\n")
            f.write(f"\n")